# Plan Implementation Audit

This notebook validates the phased plan and records results in a persistent TODO ledger that never overwrites prior entries.

Outline:
- Section 1: Repo/env helpers and quick environment printouts
- Section 2: Persistent TODO ledger (append/merge, idempotent)
- Section 3+: Verification cells for gated features, flags, and detector heuristics


# Plan Implementation Audit
This notebook validates the phased plan implementation and maintains a persistent TODO ledger without overwriting prior entries.

In [2]:
# Section 1: Locate repository root, environment, and helpers
import os, sys, json, subprocess, shutil, time, pathlib, platform
from typing import Any, Dict, List, Optional, Tuple

# In notebooks, __file__ is unavailable; walk up from CWD until we find pyproject.toml
repo_root = pathlib.Path.cwd().resolve()
while repo_root != repo_root.parent and not (repo_root / 'pyproject.toml').exists():
    repo_root = repo_root.parent
assert (repo_root / 'pyproject.toml').exists(), 'pyproject.toml not found'

# Validate expected src layout if present; do not hard-fail if restoria is absent
assert (repo_root / 'src' / 'gfpp').exists(), 'src/gfpp missing'


def run(cmd: List[str], cwd: Optional[str]=None, env: Optional[Dict[str,str]]=None) -> Tuple[int,str,str]:
    p = subprocess.Popen(cmd, cwd=cwd or str(repo_root), env={**os.environ, **(env or {})}, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    out, err = p.communicate()
    return p.returncode, out, err


def read_text(path: str) -> str:
    return pathlib.Path(path).read_text()


def write_text(path: str, data: str) -> None:
    p = pathlib.Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(data)


def read_toml(path: str) -> Dict[str, Any]:
    import tomllib as toml
    return toml.loads(read_text(path))


def write_toml(path: str, obj: Dict[str, Any]) -> None:
    try:
        import tomli_w
    except Exception:
        raise RuntimeError('tomli_w is required to write TOML in this notebook context')
    write_text(path, tomli_w.dumps(obj))


def read_yaml(path: str) -> Any:
    from ruamel.yaml import YAML
    y = YAML()
    return y.load(read_text(path))


def write_yaml(path: str, obj: Any) -> None:
    from ruamel.yaml import YAML
    y = YAML()
    s_path = pathlib.Path(path)
    s_path.parent.mkdir(parents=True, exist_ok=True)
    with s_path.open('w') as f:
        YAML().dump(obj, f)


def safe_patch_file(path: str, patch_fn):
    p = pathlib.Path(path)
    orig = p.read_text() if p.exists() else ''
    new = patch_fn(orig)
    if new != orig:
        tmp = p.with_suffix(p.suffix + '.tmp')
        tmp.write_text(new)
        tmp.replace(p)
    return p

print('Repo root:', repo_root)
print('Python:', platform.python_version())
try:
    import torch
    print('Torch:', torch.__version__, 'CUDA', getattr(torch, 'cuda', None) and torch.cuda.is_available())
except Exception as e:
    print('Torch: not available:', e)
rc, out, err = run(['uv', '--version'])
print('uv:', out.strip() if rc==0 else f'not available (rc={rc})')

Repo root: /Users/jonathanbotha/GitHub/GFPGAN
Python: 3.12.2
Torch: not available: No module named 'torch'
uv: uv 0.6.17 (8414e9f3d 2025-04-25)


In [3]:
# Section 2: Persistent TODO ledger (idempotent, append/merge; no overwrites)
import json, time, pathlib
LEDGER = repo_root / '.tools' / 'todo_ledger.json'
LEDGER.parent.mkdir(parents=True, exist_ok=True)

def load_ledger():
    if LEDGER.exists():
        try:
            return json.loads(LEDGER.read_text())
        except Exception:
            return []
    return []

def save_ledger(items):
    tmp = LEDGER.with_suffix('.json.tmp')
    tmp.write_text(json.dumps(items, indent=2))
    tmp.replace(LEDGER)

def upsert_todo(tid: int, title: str | None = None, status: str | None = None, note: str | None = None):
    items = load_ledger()
    found = None
    for it in items:
        if it.get('id') == tid:
            found = it
            break
    if not found:
        found = {'id': tid, 'title': title or '', 'status': status or 'PENDING', 'notes': [], 'updated_at': time.time()}
        items.append(found)
    if title: found['title'] = title
    if status: found['status'] = status
    if note:
        found.setdefault('notes', [])
        found['notes'].append(note)
    found['updated_at'] = time.time()
    save_ledger(items)
    return found

def record_check(tid: int, status: str, note: str):
    return upsert_todo(tid, status=status, note=note)

# Unit test for merge behavior (simple assert)
items_before = load_ledger()
_a = upsert_todo(9999, title='merge-test', status='PENDING', note='first')
_b = upsert_todo(9999, note='second')
items_after = load_ledger()
assert any(it.get('id')==9999 for it in items_after)
assert any('first' in it.get('notes', []) for it in items_after if it.get('id')==9999)
assert any('second' in it.get('notes', []) for it in items_after if it.get('id')==9999)
record_check(9999, 'PASS', 'merge works')

{'id': 9999,
 'title': 'merge-test',
 'status': 'PASS',
 'notes': ['first', 'second', 'merge works'],
 'updated_at': 1758199188.039704}

# Section 3: Plan verification checks

This section runs lightweight checks (no model downloads) and records PASS/FAIL into the persistent ledger.

We avoid heavy deps by:
- Using dry-run or internal helpers where possible
- Monkeypatching probes instead of running detectors
- Performing static-source checks when runtime validation needs unavailable dependencies


In [4]:
# Check A: CodeFormer licensing gate is enforced in CLI (static check)
from pathlib import Path
cli_src = (repo_root / 'src' / 'gfpp' / 'cli.py').read_text()
assert 'Allow non-commercial backends' in cli_src
assert 'RESTORIA_ALLOW_NONCOMMERCIAL' in cli_src
assert '--allow-noncommercial' in cli_src
print('CodeFormer licensing gate: PRESENT')
record_check(1001, 'PASS', 'CodeFormer gate strings present in gfpp/cli.py')

CodeFormer licensing gate: PRESENT


{'id': 1001,
 'title': '',
 'status': 'PASS',
 'notes': ['CodeFormer gate strings present in gfpp/cli.py'],
 'updated_at': 1758199359.154336}

In [5]:
# Check B: Precision/tiling flags are accepted and plumbed to cfg (static check)
cli_src = (repo_root / 'src' / 'gfpp' / 'cli.py').read_text()
assert '--precision' in cli_src and 'choices=["auto", "fp16", "bf16", "fp32"]' in cli_src
assert '--tile' in cli_src and '--tile-overlap' in cli_src
# Verify restorer reads precision and tile (gfpgan adapter)
rest_src = (repo_root / 'src' / 'gfpp' / 'restorers' / 'gfpgan.py').read_text()
assert 'precision = str(cfg.get("precision", "auto"))' in rest_src
assert 'tile = int(cfg.get("tile", 0))' in rest_src and 'tile_overlap' in rest_src
print('Precision/tiling flags: PRESENT and plumbed')
record_check(1002, 'PASS', 'Precision/tiling flags wired in CLI and GFPGAN restorer')

Precision/tiling flags: PRESENT and plumbed


{'id': 1002,
 'title': '',
 'status': 'PASS',
 'notes': ['Precision/tiling flags wired in CLI and GFPGAN restorer'],
 'updated_at': 1758199366.222558}

In [6]:
# Check C: Detector heuristic & mapping (planner + GFPGAN restorer)
orch_src = (repo_root / 'src' / 'gfpp' / 'core' / 'orchestrator.py').read_text()
assert 'detector_choice' in orch_src and 'params["detector"]' in orch_src
rest_src = (repo_root / 'src' / 'gfpp' / 'restorers' / 'gfpgan.py').read_text()
assert "requested_det == \"insightface\"" in rest_src and "requested_det = \"scrfd\"" in rest_src
print('Detector heuristic and insightface->scrfd mapping: PRESENT')
record_check(1003, 'PASS', 'Detector heuristic and mapping implemented')

Detector heuristic and insightface->scrfd mapping: PRESENT


{'id': 1003,
 'title': '',
 'status': 'PASS',
 'notes': ['Detector heuristic and mapping implemented'],
 'updated_at': 1758199377.991779}

In [7]:
# Check C: Face probe prefers SCRFD when available (static source presence)
probe_src = (repo_root / 'src' / 'gfpp' / 'probe' / 'faces.py').read_text()
assert '_detect_with_insightface' in probe_src and '_detect_with_facexlib' in probe_src
assert 'res = _detect_with_insightface(path)' in probe_src
print('Face probe ordering (SCRFD first): PRESENT')
record_check(1004, 'PASS', 'Face probe SCRFD preference implemented')

Face probe ordering (SCRFD first): PRESENT


{'id': 1004,
 'title': '',
 'status': 'PASS',
 'notes': ['Face probe SCRFD preference implemented'],
 'updated_at': 1758199796.451862}